In [10]:
from random import randint, uniform
from util import generate_problem, verify_witness

In [28]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def sweep_line(intervals):
    L = []
    for l, r in intervals:
        L.append( (l, 1) )
        L.append( (r, -1) )
        
    L.sort(key=lambda a: 10*a[0] - a[1])
    
    c = 0
    top = -1
    top_index = 0
    
    for index, (point, increment) in enumerate(L):
        c += increment
        if c > top:
            top = c
            top_index = index
    
    sol = int( (L[top_index][0] + L[top_index+1][0])/2 )
    
    # returns the number of constraints unsatisfied and time point
    return len(intervals) - top, sol

def walk_gene(gene, T):
    num_variables = max([max(t['i'], t['j']) for t in T])
    
    constraints_per_variable = [list() for i in range(num_variables + 1)]
    # at index i should contain availability intervals (given other variables are fixed.)
    # remark: existence of an interval is local with respect to one constraint
    # remark: so if intersection is empty, then there is no "safe" place to put this variable
    
    for c in T:
        i, j = c['i'], c['j']
        intervals = c['intervals']
        for l, r in intervals:
            # X_j - X_i in (l, r)
            # fix X_j to genes[j]
            # so genes[j] - X_i in (l, r)
            # then X_i in [X_j - r, X_j - l]
            constraints_per_variable[i].append( (gene[j] - r, gene[j] - l) )
            
            # now for j:
            constraints_per_variable[j].append( (gene[i] + l, gene[i] + r) )
            
            # ^ tested this on paper, it's ok.
    
    
    # for each constraint, sweep line, find best option
    # (later? we want to pick whatever REDUCES the unsatisfieds the most)
    # for now: pick a random variable and modify it according to sweep_line
    # if no later, then we can make the loop upstairs simpler (to only do stuff if "i" is mentioned)
    i = randint(1, num_variables)
    gene[i] = sweep_line(constraints_per_variable[i])[1]

    return gene
    
def fitness(gene, T):
    # TODO
    # different ways of computing fitness
    # like: max distance for constraint to be satisfied
    #       number of unsatisfied constraints
    #       this, but weighted somehow, etc
    
    return -len(verify_witness(gene, T))

def select(genes, retainment_ratio, T):
    genes.sort(key=lambda g: -fitness(g, T))
    return genes[: int(len(genes)*retainment_ratio+1) ]

def crossover(genes, gene_pool_size):
    while len(genes) < gene_pool_size:
        i = randint(0, len(genes)-1)
        j = randint(0, len(genes)-1)
        cross_index = randint(1, len(genes[0])-1)
        genes += [ genes[i][:cross_index] + genes[j][cross_index:] ]
        
    return genes

def mutate(T, genes, mutation_chance, r):
    for g in genes:
        if uniform(0, 1) < mutation_chance:
    # TODO:
    #        g[randint(0, len(g)-1)] = randint(-r, r)
    # or
            walk_gene(g, T)
            
    return genes
    
def genetic(T, r,
            gene_pool_size,
            retainment_ratio,
            mutation_chance,
            max_iterations):
    
    genes = [random_gene(T, r) for i in range(gene_pool_size)]
    
    for it in range(max_iterations):
        genes = select(genes, retainment_ratio, T)
        genes = crossover(genes, gene_pool_size)
        genes = mutate(T, genes, mutation_chance, r)
    
    best_gene = select(genes, 1, T)[0] # to sort genes such that first is best
    best_gene_failed = verify_witness(best_gene, T)
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [32]:
T = generate_problem(
    20,
    min_intervals=1,
    max_intervals=1,
    scaling_factor=1
)

genetic(
    T=T, r=100,
    gene_pool_size=50,
    retainment_ratio=0.1,
    mutation_chance=0.1,
    max_iterations=50,
)

best gene: [0, -99, -39, -33, -61, -73, -111, -98, -78, -90, -95, -91, -85, -95, -55, -64, -93, -46, -85, -65]
constraints failed: 7 out of: 59
failed constraints: [{'i': 0, 'j': 6, 'intervals': [(-100, 100)]}, {'i': 1, 'j': 13, 'intervals': [(-62, -23)]}, {'i': 1, 'j': 15, 'intervals': [(-92, -37)]}, {'i': 1, 'j': 16, 'intervals': [(57, 99)]}, {'i': 4, 'j': 12, 'intervals': [(-90, -87)]}, {'i': 9, 'j': 16, 'intervals': [(-97, -80)]}, {'i': 10, 'j': 15, 'intervals': [(-90, -65)]}]


## Effectiveness measurement of gene crossover

In [45]:
# problem sets:
SIMPLE = {
    'variables': 20,
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 1,
    'scaling_factor': 1,
}

BINARY = {
    'variables': 20,
    'constraint_probability': 0.25,
    'min_intervals': 2,
    'max_intervals': 2,
    'scaling_factor': 1,
}

FIVE_MAX = {
    'variables': 20,
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 5,
    'scaling_factor': 1,
}


runs = 50
run_results = []

s1, s2, s3 = 0,0,0

for _ in range(runs):
    T = generate_problem(
        **BINARY
    )
    
    g1 = random_gene(T, 100)
    g2 = random_gene(T, 100)
    
    max_flips = 50
    for _ in range(max_flips):
        g1 = walk_gene(g1, T)
        g2 = walk_gene(g2, T)
    
    
    cross_index = randint(1, len(g1) - 1)
    g3 = g1[:cross_index] + g2[cross_index:]
    
    for _ in range(max_flips):
        g1 = walk_gene(g1, T) # need to continue walking these, so that it gets a fair number of walks
        g2 = walk_gene(g2, T) # as g3 effectively has theirs + its own
        
        
        g3 = walk_gene(g3, T)
    
    r1 = verify_witness(g1, T)
    r2 = verify_witness(g2, T)
    r3 = verify_witness(g3, T)
    
    
    s1 += len(r1)
    s2 += len(r2)
    s3 += len(r3)
    
    print(len(r1))
    print(len(r2))
    print(len(r3))
    print()
    
print(s1, s2, s3)

2
3
3

7
4
6

5
2
4

6
3
5

1
3
4

3
1
4

3
3
3

3
2
3

6
7
6

2
5
3

8
5
6

3
4
4

5
4
4

2
3
2

2
3
2

3
5
3

6
5
7

6
4
1

7
6
6

5
2
3

5
5
3

4
8
10

1
5
2

12
7
11

7
6
6

6
2
6

4
6
4

12
6
13

7
7
8

5
4
7

6
1
6

4
2
3

3
4
4

3
2
0

5
7
6

2
3
5

4
3
1

14
2
9

2
2
3

6
6
5

7
3
3

7
10
9

9
9
10

0
0
0

5
1
2

8
6
5

5
4
4

8
8
9

3
3
3

3
3
4

252 209 240
